**Goal**: Analyze the competitor methods

# Competitors

In [ ]:
import project_path
from rdlib import *  # this imports the base libraries

## QSmell

In [1]:
PATH_ORIGINAL_PROGRAMS = "../data/datasets/qsmell_benchmark/original_programs"
PATH_REDUCED_PROGRAMS = "../data/datasets/qsmell_benchmark/reduced_programs"

In [4]:
from typing import List, Tuple, Dict, Any
import os
import pandas as pd


def get_number_of_LOC(folder: str, file_extension: str = ".py") -> Dict[str, int]:
    """Compute the number of LoC for each file in the folder.
    
    Note that it focuses only on files with the given extension (default: py).
    It returns a dictionary with the file name as key and the number of LoC 
    as value.
    """
    all_files = os.listdir(folder)
    all_files = [f for f in all_files if f.endswith(file_extension)]
    all_files_path = [os.path.join(folder, f) for f in all_files]
    all_contents = [open(f).read() for f in all_files_path]
    number_of_lines = [len(c.splitlines()) for c in all_contents]
    return {f: n for f, n in zip(all_files, number_of_lines)}

original_programs = get_number_of_LOC(PATH_ORIGINAL_PROGRAMS)
reduced_programs = get_number_of_LOC(PATH_REDUCED_PROGRAMS)
# remove the "wrapper_" from the keys of the reduced programs
reduced_programs = {
    k.replace("wrapper_", ""): v for k, v in reduced_programs.items()}

df_original = pd.DataFrame.from_dict(original_programs, orient="index", columns=["LOC"])
df_reduced = pd.DataFrame.from_dict(reduced_programs, orient="index", columns=["LOC"])
# merge the two dataframes
df = df_original.merge(df_reduced, left_index=True, right_index=True, suffixes=("_original", "_reduced"))
df





,LOC_original,LOC_reduced
phase_estimation.py,268,61
hhl.py,548,50
vqc.py,199,81
qsvc.py,84,52
vqd.py,798,71
ipe.py,228,61
qeom.py,598,62
mlae.py,668,87
qgan.py,489,67
ae.py,674,89
